In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import spearmanr

In [ ]:
# Text reading
ProsodyFeatures = pd.read_csv('Features\\reading_prosody_features_v1_and_v2_2025.csv')
PhonationFeatures = pd.read_csv('Features\\reading_phonation_features_v1_and_v2_2025.csv')
PhonologicalFeatures = pd.read_csv('Features\\reading_phonological_features_v1_and_v2_2025.csv')
ArticulationFeatures = pd.read_csv('Features\\reading_articulation_features_v1_and_v2_2025.csv')
GlottalFeatures = pd.read_csv('Features\\glottal_reading_features_v1_and_v2_2025.csv')

In [ ]:
# A-vowel phonation
ProsodyFeatures = pd.read_csv('Features\\a_vowel_phonation_prosody_features_v1_and_v2_2025.csv')
PhonationFeatures = pd.read_csv('Features\\a_vowel_phonation_phonation_features_v1_and_v2_2025.csv')
GlottalFeatures = pd.read_csv('Features\\glottal_phonation_features_v1_and_v2_2025.csv')

In [ ]:
EnglishMen = pd.read_csv('Population\\EnglishMen-withMT.csv', sep=';')
EnglishWomen = pd.read_csv('Population\\EnglishWomen-withMT.csv', sep=';')
FrenchMen = pd.read_csv('Population\\FrenchMen-withMT.csv', sep=';')
FrenchWomen = pd.read_csv('Population\\FrenchWomen-withMT.csv', sep=';')

In [ ]:
population = FrenchWomen.copy()
#EnglishMen
#EnglishWomen
#FrenchMen
#FrenchWomen
population

In [ ]:
featureset = GlottalFeatures.copy()
#ProsodyFeatures
#PhonationFeatures
#ArticulationFeatures
#PhonologicalFeatures
#GlottalFeatures

In [ ]:
featureset = featureset.iloc[:, 1:]
featureset['id'] = featureset['id'].str.extract(r'(^\d+-\d+)')
featureset = featureset[featureset['id'].notna()]
# remove kurtosis, skewness, min and max
featureset = featureset.drop(columns=featureset.filter(regex='ku|sk|min|max').columns)
featureset

In [ ]:
# Join population and feature set
Features = featureset.merge(population, left_on='id', right_on='UniqueId', how='inner')
Features

In [ ]:
# Removing redundant features
# This code was written by Bour, C. & Elbeji, A.
def calculate_vif(df):
    """Computes the Variance Inflation Factor (VIF) for each feature in the dataset."""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

def get_highly_correlated_groups(df, threshold=0.90, alpha=0.05):
    """Identifies groups of highly correlated features and returns them as a list of sets."""
    correlation_matrix = df.corr(method='spearman')
    p_value_matrix = df.corr(method=lambda x, y: spearmanr(x, y)[1])

    correlated_groups = []
    visited = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            p_value = p_value_matrix.iloc[i, j]
            if abs(correlation_matrix.iloc[i, j]) > threshold and p_value < alpha:
                feature1 = correlation_matrix.columns[i]
                feature2 = correlation_matrix.columns[j]

                # Find existing group that contains one of these features
                found_group = None
                for group in correlated_groups:
                    if feature1 in group or feature2 in group:
                        found_group = group
                        break
                
                if found_group:
                    found_group.add(feature1)
                    found_group.add(feature2)
                else:
                    correlated_groups.append({feature1, feature2})

    return correlated_groups

def iterative_feature_selection(df, correlation_threshold=0.90, vif_threshold=5, alpha=0.05, max_iterations=100):
    """Iteratively removes highly correlated and high VIF features while keeping at least one."""
    df_filtered = df.copy()
    iteration = 0

    while df_filtered.shape[1] > 1 and iteration < max_iterations:
        features_before = set(df_filtered.columns)

        # Step 1: Identify and remove highly correlated features while keeping one from each group
        correlated_groups = get_highly_correlated_groups(df_filtered, correlation_threshold, alpha)
        if correlated_groups:
            vif_values = calculate_vif(df_filtered).set_index("Feature")

            for group in correlated_groups:
                if len(group) > 1:
                    # Keep the feature with the lowest VIF
                    group_vif = vif_values.loc[list(group)]
                    feature_to_keep = group_vif["VIF"].idxmin()
                    group.remove(feature_to_keep)
                    df_filtered = df_filtered.drop(columns=group)

        # Step 2: Remove features with high VIF iteratively
        vif_values = calculate_vif(df_filtered)
        while vif_values["VIF"].max() > vif_threshold:
            worst_vif_feature = vif_values.sort_values(by="VIF", ascending=False).iloc[0]["Feature"]
            df_filtered = df_filtered.drop(columns=[worst_vif_feature])
            vif_values = calculate_vif(df_filtered)

        # Stop if no features were removed in this iteration
        features_after = set(df_filtered.columns)
        if features_before == features_after:
            break

        iteration += 1

    print(f"\n🏁 Final feature count: {df_filtered.shape[1]}")
    return df_filtered

print("----------- Removing features -----------")
features_filtered = Features.drop("id", axis=1).set_index("UniqueId")
features_to_select = features_filtered.iloc[:, :-10]
selected_features = iterative_feature_selection(features_to_select)
Features = selected_features.join(features_filtered.iloc[:, -10:])

In [ ]:
# Data quality checks
# The removal of participants with >75% of missing data was carried out at a previous step
print(Features[Features.isna().any(axis=1)])
print(Features[(Features == 0).sum(axis=1) > 5])

## Regressions

In [ ]:
selected_features = Features.columns[:-10]

# Define predictor groups
stress_predictor = ["stress_score"]
base_adjustment = ["age", "educ", "alc", "smk_1", "chronicD", "mother-tongue"]
full_adjustment = base_adjustment + ["who5", "phq9", "fss"]

# Standardize numerical predictors (except binary/categorical variables)
scaler = StandardScaler()
scaled_features = Features.copy()
scaled_features[["stress_score", "age", "alc", "who5", "phq9", "fss"]] = scaler.fit_transform(
    Features[["stress_score", "age", "alc", "who5", "phq9", "fss"]]
)

# Encode categorical variables
scaled_features = pd.get_dummies(scaled_features, columns=["educ", "smk_1", "chronicD", "mother-tongue"], drop_first=True)

# Update predictor lists to reflect new column names
categorical_vars = ["educ_higher", "smk_1_Smoker", "chronicD_yes", "mother-tongue_other"]
base_adjustment = ["age", "alc"] + categorical_vars
full_adjustment = base_adjustment + ["who5", "phq9", "fss"]

# Prepare results storage
results = []

# Loop through each selected feature
for feature in selected_features:
    # Check for NaNs and 0s in the feature
    if Features[feature].isna().sum() > 0:
        print(f"Warning: {feature} has missing values. Consider handling them.")
    if (Features[feature] == 0).sum() > 0:
        print(f"Warning: {feature} has zero values.")

    # Standardize the dependent variable
    y = (scaled_features[feature] - scaled_features[feature].mean()) / scaled_features[feature].std()

    # Define models
    models = [
        ("Model 1", stress_predictor),
        ("Model 2", stress_predictor + base_adjustment),
        ("Model 3", stress_predictor + full_adjustment)
    ]

    # Run regressions
    for model_name, predictors in models:
        X = scaled_features[predictors].astype(float)
        X = sm.add_constant(X)  # Add intercept
        
        # Fit OLS model
        model = sm.OLS(y, X).fit()
        
        # Store results
        ci_values = model.conf_int()
        for param, coef, pval in zip(model.params.index, model.params, model.pvalues):
                if param == "const":
                    continue  # skip intercept
                results.append({
                    "Feature": feature,
                    "Model": model_name,
                    "Predictor": param,
                    "Coefficient": coef,
                    "CI Lower": ci_values.loc[param, 0],
                    "CI Upper": ci_values.loc[param, 1],
                    "p-value": pval
                })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Create a new column for FDR-corrected p-values, defaulting to NaN
results_df["FDR-adjusted p"] = np.nan

# Apply FDR correction within each model for stress_score only
for model in results_df["Model"].unique():
    mask = (results_df["Model"] == model) & (results_df["Predictor"] == "stress_score")
    pvals = results_df.loc[mask, "p-value"]
    if not pvals.empty:
        corrected = multipletests(pvals, method="fdr_bh")[1]
        results_df.loc[mask, "FDR-adjusted p"] = corrected

# Show final results
print(results_df)

In [ ]:
results_df.to_csv('resultats-disvoice.csv', sep=',', index=False)

### Formatting results in Tables S3 and S4 showing the progressive adjustments

In [ ]:
df_stress = results_df[results_df['Predictor'] == 'stress_score'].copy()

# Format the coefficient and confidence interval
df_stress['formatted_result'] = df_stress.apply(
    lambda row: f"{row['Coefficient']:.2f} [{row['CI Lower']:.2f}, {row['CI Upper']:.2f}]",
    axis=1
)

# Extract model number (assuming 'Model 1', 'Model 2', etc.)
df_stress['Model_Num'] = df_stress['Model'].str.extract(r'(\d+)')

# Pivot to wide format with one row per Feature
result = df_stress.pivot(index='Feature', columns='Model_Num', values=['formatted_result', 'FDR-adjusted p'])

# Flatten MultiIndex columns
result.columns = [
    f"M{col[1]}" if col[0] == 'formatted_result' else f"pvalue{col[1]}"
    for col in result.columns
]

# Optional: sort columns to desired order
ordered_cols = []
for i in range(1, 4):  # for Model 1 to Model 3
    if f"M{i}" in result.columns and f"pvalue{i}" in result.columns:
        ordered_cols.extend([f"M{i}", f"pvalue{i}"])
result = result[ordered_cols]

# Reset index to bring Feature back as a column
result = result.reset_index()

# Display or return result
print(result)
result.to_csv('resultats-disvoice-progressive.csv', sep=',', index=False)